In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip3 install transformers

In [3]:
import json
import pickle
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from transformers import AutoTokenizer
import torch

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
df = pd.read_csv("https://huggingface.co/datasets/segyges/openwordnet-categoricals/resolve/main/openwordnet-categoricals.csv", keep_default_na=False)
df_pos = df[['members', 'a', 'n', 'r', 's', 'v']]
df_pos.head()

,members,a,n,r,s,v
0,.22,0,1,0,0,0
1,.22 caliber,1,0,0,0,0
2,.22 calibre,1,0,0,0,0
3,.22-caliber,1,0,0,0,0
4,.22-calibre,1,0,0,0,0


In [6]:
df_pos.isnull().sum()

members    0
a          0
n          0
r          0
s          0
v          0
dtype: int64

In [7]:
df_pos[['a', 'n', 'r', 's', 'v']].value_counts()

a  n  r  s  v
0  1  0  0  0    116704
   0  0  1  0     11366
         0  1      7340
1  0  0  0  0      5275
0  0  1  0  0      3901
   1  0  0  1      3701
1  0  0  1  0      1442
0  1  0  1  0      1230
1  1  0  0  0      1007
         1  0       343
0  0  1  1  0       222
   1  0  1  1       220
   0  0  1  1       103
   1  1  1  0        70
1  1  0  1  1        68
0  1  1  0  0        65
1  0  1  1  0        43
         0  0        37
   1  1  1  1        34
   0  0  1  1        30
   1  0  0  1        30
      1  1  0        30
         0  0        22
   0  0  0  1        21
0  1  1  1  1        18
         0  1        18
1  0  1  1  1        11
0  0  1  0  1         6
         1  1         3
1  1  1  0  1         1
Name: count, dtype: int64

In [8]:
model_name = "EleutherAI/pythia-12b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
[key for key in tokenizer.vocab.keys() if "dog" in key.lower()]

['Dog', 'ĠDog', 'Ġendogenous', 'dog', 'Ġdog', 'Ġdogs', 'ĠDogs']

In [10]:
[ord(key[0]) for key in tokenizer.vocab.keys() if "dog" in key.lower()]

[68, 288, 288, 100, 288, 288, 288]

In [11]:
def get_targets(words, embeddings, tokenizer):
  X, y, terms = [], [], []
  for i, row in words.iterrows():
    lowercased = row['members'].lower()
    capitalized = row['members'].capitalize()
    leading_space = chr(288)
    options = [lowercased, capitalized, leading_space + lowercased, leading_space + capitalized]
    for option in options:
      ids = tokenizer(option, add_special_tokens=False, padding=False)['input_ids']
      if len(ids) == 1:
        embedding_idx = torch.tensor(ids, device=device)[0]
        token = embeddings[embedding_idx]
        X.append(token)
        y.append(row.drop('members').values.astype(int))
        terms.append(option)
  return X, y, terms

In [12]:
def eval_preds(y_test, y_pred):
    """
    Takes in (y_test, y_pred) and outputs precision, recall, f1, average precision, average recall, ROC_AUC and PR_AUC.
    But Gyges ran out of steam so Clyde here taking over to do what i do best. ctrl+c ctrl+v code and take credit.
    :)
    """

    n_classes = 5

    # Convert y_test and y_pred to numpy arrays if they are not already
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_pred_onehot = (y_pred >= 0.5).astype(int)

    # Average precision score for all classes
    micro_precision = average_precision_score(y_test, y_pred, average='micro')

    # Calculate average precision score for each class separately
    classwise_avg_precision = average_precision_score(y_test, y_pred, average=None)

    # Calculate recall score for each class separately
    classwise_recall = recall_score(y_test, y_pred_onehot, average=None)

    # Calculate micro-averaged recall
    micro_recall = recall_score(y_test, y_pred_onehot, average='micro')

    # PR AUC + ROC AUC
    # Calculate Precision-Recall curve and area under the curve (PR AUC) for each class separately
    precision = dict()
    recall = dict()
    pr_auc = dict()

    for i in range(n_classes):  # Assuming n_classes is the number of classes
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
        pr_auc[i] = auc(recall[i], precision[i])

    # Calculate ROC curve and area under the curve (ROC AUC) for each class separately
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):  # Assuming n_classes is the number of classes
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = roc_auc_score(y_test[:, i], y_pred[:, i])

    df_dict = {"avg precision per class": list(classwise_avg_precision),
               "avg recall per class": list(classwise_recall),
               "PR AUC": list(pr_auc.values()),
               "ROC AUC": list(roc_auc.values()),
               "micro precision": [micro_precision],
               "micro recall": [micro_recall]}
    return(df_dict)

In [13]:
all_results = {}

In [14]:
# Just testing that this works a few times to work out any oddities
# count = 0
# for key, value in data.items():
#     print(key)
#     print(value.shape)
#     count += 1
#     if count == 5:
#         break

#     X, y, terms = get_targets(df_pos, value, tokenizer)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1337)

#     classifier = MultiOutputClassifier(make_pipeline(StandardScaler(), SVC(kernel='sigmoid', probability=True)))
#     y_pred_proba = classifier.fit(X_train, y_train).predict_proba(X_test)
#     # Bunch of numpy fiddling to get the output of predict_proba into the correct format
#     output_array = np.stack(y_pred_proba)
#     output_transposed = np.transpose(output_array, (1, 0, 2))
#     reshaped_array = output_transposed.reshape(output_transposed.shape[0], -1)
#     positive_probs = reshaped_array[:, 1::2]
#     results = eval_preds(y_test, positive_probs)
#     print(results)

In [ ]:
files = [
 './pythia-12b-weights/embed_only_0-29000.pkl',
 './pythia-12b-weights/embed_only_30000-69000.pkl',
 './pythia-12b-weights/embed_only_70000-109000.pkl',
 './pythia-12b-weights/embed_only_110000-143000.pkl'
]

output = {}
for filename in files:
    with open(filename, 'rb') as in_file:
        data = pickle.load(in_file)
        for key, value in data.items():
            X, y, terms = get_targets(df_pos, value, tokenizer)
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1337)
        
            classifier = MultiOutputClassifier(make_pipeline(StandardScaler(), SVC(kernel='sigmoid', probability=True)))
            y_pred_proba = classifier.fit(X_train, y_train).predict_proba(X_test)
            # Bunch of numpy fiddling to get the output of predict_proba into the correct format
            output_array = np.stack(y_pred_proba)
            output_transposed = np.transpose(output_array, (1, 0, 2))
            reshaped_array = output_transposed.reshape(output_transposed.shape[0], -1)
            positive_probs = reshaped_array[:, 1::2]
            results = eval_preds(y_test, positive_probs)
            all_results[key] = results
            # Save to disk
            serialized = json.dumps(all_results)
            with open('checkpointed_data.json', 'w') as out_file:
                out_file.write(serialized)
        del(data)